# Day 5: Hydrothermal Venture

[*Advent of Code 2021 day 5*](https://adventofcode.com/2021/day/5) and [*solution megathread*](https://redd.it/r9824c)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2021/05/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2021%2F05%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys

sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

Today was feeling a bit grinding - I felt as if this could have been written using a few lines of numpy Python, but at least as I could factor out functions like `dim_to_coords` and not do everything once per x and y or per from-coordinate and to-coordinate, it felt slightly less verbose. Predictably, I've rarely encountered as many off-by-one or accidentally-swapped xs and ys as in this problem. Fortunately, Part Two became really simple, as I already was practically able to handle also diagonal lines. Small blessings... onward.

In [5]:
testdata = """0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
Coord = tuple[int, int]
VentLine = tuple[Coord, Coord]
VentMap = list[list[int]]


def parse_data(data: list[str]) -> list[VentLine]:
    result = []
    for line in data:
        cf, ct = line.split(' -> ')
        x1, y1 = tuple(map(int, cf.split(',')))
        x2, y2 = tuple(map(int, ct.split(',')))
        result.append(((x1, y1), (x2, y2)))
    return result


def ventmap_to_string(ventmap: VentMap) -> str:
    rows = []
    for row in ventmap:
        rows.append(''.join(str(c) if c != 0 else '.' for c in row))
    return '\n'.join(rows)


def filter_orthogonal(ventlines: list[VentLine]) -> list[VentLine]:
    # What is the right way to unpack tuples for use in a filter?
    def orth_filter(cp):
        cf, ct = cp
        return cf[0] == ct[0] or cf[1] == ct[1]
    return filter(orth_filter, ventlines)


def dim_to_coords(df: int, dt: int, length: int) -> list[int]:
    if df == dt:
        return [df] * length

    assert(abs(df - dt) + 1 == length)
    if dt < df:
        return list(range(df, dt - 1, -1))
    else:
        return list(range(df, dt + 1, 1))


def line_to_coords(line: VentLine) -> list[Coord]:
    cf, ct = line
    length = max(abs(cf[0] - ct[0]),
                 abs(cf[1] - ct[1])) + 1
    xs = dim_to_coords(cf[0], ct[0], length)
    ys = dim_to_coords(cf[1], ct[1], length)
    return list(zip(xs, ys))


def lines_to_coords(lines: list[VentLine]) -> list[Coord]:
    result = []
    for line in lines:
        result += line_to_coords(line)
    return result


def allocate_map(coords: list[Coord]) -> VentMap:
    size = max(max(coords, key=lambda c: c[0])[0],
               max(coords, key=lambda c: c[1])[1]) + 1
    zero_row = [0] * size
    return [zero_row[:] for _ in range(size)]


def note_coords(ventmap: VentMap, coords: list[Coord]):
    for cx, cy in coords:
        ventmap[cy][cx] += 1


def count_overlaps(ventmap: VentMap) -> int:
    return sum(sum(c > 1 for c in row)
               for row in ventmap)


def my_part1_solution(data: list[str],
                      debug: bool = False) -> int:
    ventlines = parse_data(data)
    lines_orth = filter_orthogonal(ventlines)
    coords_orth = lines_to_coords(lines_orth)
    ventmap = allocate_map(coords_orth)
    note_coords(ventmap, coords_orth)
    if debug:
        print(ventmap_to_string(ventmap))
    return count_overlaps(ventmap)

In [7]:
assert(my_part1_solution(testdata, debug=True) == 5)

.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....


In [8]:
my_part1_solution(inputdata)

4826

In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

In [11]:

def my_part2_solution(data: list[str],
                      debug: bool = False) -> int:
    ventlines = parse_data(data)
    coords = lines_to_coords(ventlines)
    ventmap = allocate_map(coords)
    note_coords(ventmap, coords)
    if debug:
        print(ventmap_to_string(ventmap))
    return count_overlaps(ventmap)

In [12]:
assert(my_part2_solution(testdata, debug=True) == 12)

1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....


In [13]:
my_part2_solution(inputdata)

16793

In [14]:
HTML(downloaded['part2_footer'])